In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DT').getOrCreate()


In [ ]:
df = spark.read.format("csv").option("sep",",").option("inferSchema","true").option("header","true").load("test2_score.txt")
df.printSchema()

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()
numeric_features = ['Userid','trackid','label','albumscore','artistscore','genreamax','genreamin','genreamean']

df.select(numeric_features).describe().toPandas().transpose()

In [ ]:
# import matplotlib.pyplot as plt
# numeric_data = df.select(numeric_features).toPandas()

# axs = pd.plotting.scatter_matrix(numeric_data, figsize=(8, 8));
# n = len(numeric_data.columns)
# for i in range(n):
#     v = axs[i, 0]
#     v.yaxis.label.set_rotation(0)
#     v.yaxis.label.set_ha("right")
#     v.set_yticks(())
#     h = axs[n-1, i]
#     h.xaxis.label.set_rotation(90)
#     h.set_xticks(())
# plt.show()


In [ ]:
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.types as types
df=df.withColumn('label',df['label'].cast(types.IntegerType()))
df=df.withColumn('Userid',df['Userid'].cast(types.IntegerType()))
df=df.withColumn('trackid',df['trackid'].cast(types.IntegerType()))
df=df.withColumn('albumscore',df['albumscore'].cast(types.IntegerType()))
df=df.withColumn('artistscore',df['artistscore'].cast(types.IntegerType()))
df=df.withColumn('genreamax',df['genreamax'].cast(types.IntegerType()))
df=df.withColumn('genreamin',df['genreamin'].cast(types.IntegerType()))
df=df.withColumn('genreamean',df['genreamean'].cast(types.IntegerType()))

assembler=VectorAssembler(inputCols=['Userid','trackid','label','artistscore','genreamax','genreamin','genreamean'],outputCol='features')


In [ ]:
from pyspark.ml import Pipeline
pipeline=Pipeline(stages=[assembler])
model=pipeline.fit(df)
df=model.transform(df)


In [ ]:
train,test=df.randomSplit([0.95,0.05])

In [ ]:
handleInvalid = "keep"
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label',maxDepth = 3,)
dtModel = dt.fit(train)

In [ ]:
predictions = dtModel.transform(test)
predictions.show(10)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

In [ ]:
df1 = spark.read.format("csv").option("sep",",").option("inferSchema","true").option("header","true").load("test_score_1.txt")
df1.printSchema()

In [ ]:
import pandas as pd
pd.DataFrame(df1.take(5), columns=df1.columns).transpose()
numeric_features = ['Userid','trackid','albumscore','artistscore','genreamax','genreamin','genreamean']

df1.select(numeric_features).describe().toPandas().transpose()

In [ ]:
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.types as types
# df1=df1.withColumn('label',df['label'].cast(types.IntegerType()))
df1=df1.withColumn('Userid',df1['Userid'].cast(types.IntegerType()))
df1=df1.withColumn('trackid',df1['trackid'].cast(types.IntegerType()))
df1=df1.withColumn('albumscore',df1['albumscore'].cast(types.IntegerType()))
df1=df1.withColumn('artistscore',df1['artistscore'].cast(types.IntegerType()))
df1=df1.withColumn('genreamax',df1['genreamax'].cast(types.IntegerType()))
df1=df1.withColumn('genreamin',df1['genreamin'].cast(types.IntegerType()))
df1=df1.withColumn('genreamean',df1['genreamean'].cast(types.IntegerType()))

assembler1=VectorAssembler(inputCols=['Userid','trackid','artistscore','genreamax','genreamin','genreamean'],outputCol='features')



In [ ]:
from pyspark.ml import Pipeline
pipeline=Pipeline(stages=[assembler1])
model1=pipeline.fit(df1)
df1=model1.transform(df1)

In [ ]:
predictions1 = dtModel.transform(df1)
predictions1.show(10)

In [ ]:
predictions1.count()

In [ ]:
pre=predictions1.rdd.map(lambda x: x.prediction).collect()

In [ ]:
user=predictions1.rdd.map(lambda x: x.Userid).collect()

In [ ]:
track=predictions1.rdd.map(lambda x: x.trackid).collect()

In [ ]:
track

In [ ]:
output = []
for i in range(120000):
    output.append([user[i],track[i],pre[i]])

In [ ]:
output

In [ ]:
import csv
with open("decisionTreeOutput.csv", "w") as output_file:
    
    writer = csv.writer(output_file)
    writer.writerow(["TrackID", "Predictor"])
    for line in output:
        item = [str(line[0]) + "_" + str(line[1]), line[2]]
        writer.writerow(item)


In [ ]:
for line in output:
    string = line[0] + "," +

# 